In [4]:
import json
import os
from PIL import Image
import re
import pandas as pd
from ast import literal_eval as make_tuple

In [5]:
with open('datasamplet.json', 'r') as f:
    data = json.load(f)

In [6]:
ddir = "ite2/train/images/"
arr = [f for f in os.listdir(ddir) if re.match(r'.*\.(jpg|JPG)', f)]

In [7]:
for i in arr:
    path = ddir + str(i)
    im = Image.open(path)
    width, height = im.size
    name = i.replace(".jpg","") 
    value = {'license': 0, 'file_name': i, 'width': width, 'height': height, 'id': name}
    data['images'].append(value)
len(data['images'])

515

In [8]:
annodata = pd.read_csv('ite2/train_list.csv')
annodatawobg = annodata[~annodata["seg_name"].str.contains("_a")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("MET43_2_0001_s")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("VA01_1_0001_s")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("VA30_3_0007_s")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("VA07_1_0005_s")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("LACMA05_1_0008_t")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("KOC03_3_0002_c")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("IPOT04_1_0001_c")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("TLCM16_3_0002_t")]
annodatawobg = annodatawobg[~annodatawobg["seg_name"].str.contains("TLCM15_1_0000_c")]


/Users/yuchen/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (34,35,36,37,38,39,75,76,77,78,79,80,116,117,118,119,120,121) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
for index, row in annodatawobg.iterrows():
    path = "ite2/train/ct/"+row["seg_name"]+".csv"
    segcord = pd.read_csv(path)
    segcord4 = segcord[::20]
    if row["seg_name"][-1:] == "c":
        t = 2
    elif row["seg_name"][-1:] == "t":
        t = 3
    elif row["seg_name"][-1:] == "s":
        t = 1
    bboxlist=make_tuple(row["bbox(y1,x1,y2,x2)"])
    xmax=max(segcord["x"])
    xmin=min(segcord["x"])
    ymax=max(segcord["y"])
    ymin=min(segcord["y"])
    w=xmax-xmin
    l=ymax-ymin
    realbbox = list((xmin-w*.2,ymin-l*.2,w*1.4,l*1.4))
#     realbbox2 = list((bboxlist[1]*.9,bboxlist[0]*.9,(bboxlist[3]-bboxlist[1])*1.2,(bboxlist[2]-bboxlist[0])*1.2))
#     print("realbbox11111")
#     print(realbbox)
#     print("realbbox22222")
#     print(realbbox2)
    
    seglist = []
    for i in range(len(segcord)):
        x = segcord["x"][i]
        y = segcord["y"][i]
        seglist.append(x)
        seglist.append(y)
    annovalue = {
    'segmentation': [seglist],
    'iscrowd': 0,
    'image_id': row["name"],
    'category_id': t,
    'id': index+1,
    'bbox': realbbox,
    'area': row["area"]
    }
    data['annotations'].append(annovalue)
len(data['annotations'])


3806

In [10]:
data["categories"]

[{'supercategory': 'motif', 'id': 1, 'name': 'saz'},
 {'supercategory': 'motif', 'id': 2, 'name': 'carnation'},
 {'supercategory': 'motif', 'id': 3, 'name': 'tulip'}]

In [11]:
with open('ite2/train/train.json', 'w') as f:
    json.dump(data, f)